In [1]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import TfidfVectorizer
import difflib
import pandas as pd
import numpy as np

In [2]:
df = pd.read_csv('movies_metadata.csv',low_memory=False)
df1 = pd.read_csv('credits.csv')
df2 = pd.read_csv('keywords.csv')
df3 = pd.read_csv('links_small.csv')
df4 = pd.read_csv('ratings_small.csv')

In [3]:
df_mod = df

In [4]:
df_mod.dropna(subset=['imdb_id','runtime','title','release_date'],inplace=True)

In [5]:
df_mod['id'] = [int(s) for s in df_mod['id']]
import ast
# s = (df['genres'].iloc[0])
def gen(s):
    ls = ast.literal_eval(s)
    h = ''
    for ele in ls:
        if h == '':
            h = h + ele['name']
        else:
            h = h + ' ' + ele['name']
    return h

In [6]:
df_mod['genres'] = [gen(s) for s in df_mod['genres']]


In [7]:
df_mod['adult'] = [0 if s == 'False' else 1 for s in df_mod['adult']]
df_mod['release_date'] = [int(s[:4]) for s in df_mod['release_date']]
df_mod = df_mod.rename(columns={'release_date':'release_year'})

In [8]:
df_mod.head()

,adult,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,overview,...,release_year,revenue,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count
0,0,"{'id': 10194, 'name': 'Toy Story Collection', ...",30000000,Animation Comedy Family,http://toystory.disney.com/toy-story,862,tt0114709,en,Toy Story,"Led by Woody, Andy's toys live happily in his ...",...,1995,373554033.0,81.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,NaN,Toy Story,False,7.7,5415.0
1,0,NaN,65000000,Adventure Fantasy Family,NaN,8844,tt0113497,en,Jumanji,When siblings Judy and Peter discover an encha...,...,1995,262797249.0,104.0,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,Roll the dice and unleash the excitement!,Jumanji,False,6.9,2413.0
2,0,"{'id': 119050, 'name': 'Grumpy Old Men Collect...",0,Romance Comedy,NaN,15602,tt0113228,en,Grumpier Old Men,A family wedding reignites the ancient feud be...,...,1995,0.0,101.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Still Yelling. Still Fighting. Still Ready for...,Grumpier Old Men,False,6.5,92.0
3,0,NaN,16000000,Comedy Drama Romance,NaN,31357,tt0114885,en,Waiting to Exhale,"Cheated on, mistreated and stepped on, the wom...",...,1995,81452156.0,127.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Friends are the people who let you be yourself...,Waiting to Exhale,False,6.1,34.0
4,0,"{'id': 96871, 'name': 'Father of the Bride Col...",0,Comedy,NaN,11862,tt0113041,en,Father of the Bride Part II,Just when George Banks has recovered from his ...,...,1995,76578911.0,106.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Just When His World Is Back To Normal... He's ...,Father of the Bride Part II,False,5.7,173.0


In [9]:
df_mod = df_mod.merge(df1,on='id')

In [10]:
def director(s):
    ls = ast.literal_eval(s)
    for i in ls:
        if i['job'] == 'Director':
            return i['name']
    return None
df_mod['Director'] = [director(s) for s in df_mod['crew']]

In [11]:
def allCast(s):
    ls = ast.literal_eval(s)
#     return ls
    h = ''
    for i in ls:
        p = i['name']
        if h == '': 
            h = h + p
        else:
            h = h + ' ' + p
    return h
df_mod['cast'] = [allCast(s) for s in df_mod['cast']]

In [12]:
df_mod = df_mod.drop(['crew'],axis=1)

In [13]:
df_mod = df_mod.merge(df2,on='id')
df_mod.head()

,adult,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,overview,...,spoken_languages,status,tagline,title,video,vote_average,vote_count,cast,Director,keywords
0,0,"{'id': 10194, 'name': 'Toy Story Collection', ...",30000000,Animation Comedy Family,http://toystory.disney.com/toy-story,862,tt0114709,en,Toy Story,"Led by Woody, Andy's toys live happily in his ...",...,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,NaN,Toy Story,False,7.7,5415.0,Tom Hanks Tim Allen Don Rickles Jim Varney Wal...,John Lasseter,"[{'id': 931, 'name': 'jealousy'}, {'id': 4290,..."
1,0,NaN,65000000,Adventure Fantasy Family,NaN,8844,tt0113497,en,Jumanji,When siblings Judy and Peter discover an encha...,...,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,Roll the dice and unleash the excitement!,Jumanji,False,6.9,2413.0,Robin Williams Jonathan Hyde Kirsten Dunst Bra...,Joe Johnston,"[{'id': 10090, 'name': 'board game'}, {'id': 1..."
2,0,"{'id': 119050, 'name': 'Grumpy Old Men Collect...",0,Romance Comedy,NaN,15602,tt0113228,en,Grumpier Old Men,A family wedding reignites the ancient feud be...,...,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Still Yelling. Still Fighting. Still Ready for...,Grumpier Old Men,False,6.5,92.0,Walter Matthau Jack Lemmon Ann-Margret Sophia ...,Howard Deutch,"[{'id': 1495, 'name': 'fishing'}, {'id': 12392..."
3,0,NaN,16000000,Comedy Drama Romance,NaN,31357,tt0114885,en,Waiting to Exhale,"Cheated on, mistreated and stepped on, the wom...",...,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Friends are the people who let you be yourself...,Waiting to Exhale,False,6.1,34.0,Whitney Houston Angela Bassett Loretta Devine ...,Forest Whitaker,"[{'id': 818, 'name': 'based on novel'}, {'id':..."
4,0,"{'id': 96871, 'name': 'Father of the Bride Col...",0,Comedy,NaN,11862,tt0113041,en,Father of the Bride Part II,Just when George Banks has recovered from his ...,...,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Just When His World Is Back To Normal... He's ...,Father of the Bride Part II,False,5.7,173.0,Steve Martin Diane Keaton Martin Short Kimberl...,Charles Shyer,"[{'id': 1009, 'name': 'baby'}, {'id': 1599, 'n..."


In [14]:
def getKeys(s):
    ls = ast.literal_eval(s)
#     return ls
    h = ''
    for i in ls:
        p = i['name']
        if h == '': 
            h = h + p
        else:
            h = h + ' ' + p
    return h
df_mod['keywords'] = [getKeys(s) for s in df_mod['keywords']]

In [15]:
df_mod.head()

,adult,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,overview,...,spoken_languages,status,tagline,title,video,vote_average,vote_count,cast,Director,keywords
0,0,"{'id': 10194, 'name': 'Toy Story Collection', ...",30000000,Animation Comedy Family,http://toystory.disney.com/toy-story,862,tt0114709,en,Toy Story,"Led by Woody, Andy's toys live happily in his ...",...,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,NaN,Toy Story,False,7.7,5415.0,Tom Hanks Tim Allen Don Rickles Jim Varney Wal...,John Lasseter,jealousy toy boy friendship friends rivalry bo...
1,0,NaN,65000000,Adventure Fantasy Family,NaN,8844,tt0113497,en,Jumanji,When siblings Judy and Peter discover an encha...,...,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,Roll the dice and unleash the excitement!,Jumanji,False,6.9,2413.0,Robin Williams Jonathan Hyde Kirsten Dunst Bra...,Joe Johnston,board game disappearance based on children's b...
2,0,"{'id': 119050, 'name': 'Grumpy Old Men Collect...",0,Romance Comedy,NaN,15602,tt0113228,en,Grumpier Old Men,A family wedding reignites the ancient feud be...,...,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Still Yelling. Still Fighting. Still Ready for...,Grumpier Old Men,False,6.5,92.0,Walter Matthau Jack Lemmon Ann-Margret Sophia ...,Howard Deutch,fishing best friend duringcreditsstinger old men
3,0,NaN,16000000,Comedy Drama Romance,NaN,31357,tt0114885,en,Waiting to Exhale,"Cheated on, mistreated and stepped on, the wom...",...,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Friends are the people who let you be yourself...,Waiting to Exhale,False,6.1,34.0,Whitney Houston Angela Bassett Loretta Devine ...,Forest Whitaker,based on novel interracial relationship single...
4,0,"{'id': 96871, 'name': 'Father of the Bride Col...",0,Comedy,NaN,11862,tt0113041,en,Father of the Bride Part II,Just when George Banks has recovered from his ...,...,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Just When His World Is Back To Normal... He's ...,Father of the Bride Part II,False,5.7,173.0,Steve Martin Diane Keaton Martin Short Kimberl...,Charles Shyer,baby midlife crisis confidence aging daughter ...


In [16]:
df_mod.shape
df_mod.isnull().sum()
df_mod.head()

,adult,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,overview,...,spoken_languages,status,tagline,title,video,vote_average,vote_count,cast,Director,keywords
0,0,"{'id': 10194, 'name': 'Toy Story Collection', ...",30000000,Animation Comedy Family,http://toystory.disney.com/toy-story,862,tt0114709,en,Toy Story,"Led by Woody, Andy's toys live happily in his ...",...,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,NaN,Toy Story,False,7.7,5415.0,Tom Hanks Tim Allen Don Rickles Jim Varney Wal...,John Lasseter,jealousy toy boy friendship friends rivalry bo...
1,0,NaN,65000000,Adventure Fantasy Family,NaN,8844,tt0113497,en,Jumanji,When siblings Judy and Peter discover an encha...,...,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,Roll the dice and unleash the excitement!,Jumanji,False,6.9,2413.0,Robin Williams Jonathan Hyde Kirsten Dunst Bra...,Joe Johnston,board game disappearance based on children's b...
2,0,"{'id': 119050, 'name': 'Grumpy Old Men Collect...",0,Romance Comedy,NaN,15602,tt0113228,en,Grumpier Old Men,A family wedding reignites the ancient feud be...,...,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Still Yelling. Still Fighting. Still Ready for...,Grumpier Old Men,False,6.5,92.0,Walter Matthau Jack Lemmon Ann-Margret Sophia ...,Howard Deutch,fishing best friend duringcreditsstinger old men
3,0,NaN,16000000,Comedy Drama Romance,NaN,31357,tt0114885,en,Waiting to Exhale,"Cheated on, mistreated and stepped on, the wom...",...,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Friends are the people who let you be yourself...,Waiting to Exhale,False,6.1,34.0,Whitney Houston Angela Bassett Loretta Devine ...,Forest Whitaker,based on novel interracial relationship single...
4,0,"{'id': 96871, 'name': 'Father of the Bride Col...",0,Comedy,NaN,11862,tt0113041,en,Father of the Bride Part II,Just when George Banks has recovered from his ...,...,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Just When His World Is Back To Normal... He's ...,Father of the Bride Part II,False,5.7,173.0,Steve Martin Diane Keaton Martin Short Kimberl...,Charles Shyer,baby midlife crisis confidence aging daughter ...


In [36]:
dff =df_mod

In [37]:
dff['popularity'].describe()

count    46275.000000
mean         2.914243
std          5.968169
min          0.000000
25%          0.392888
50%          1.129734
75%          3.660012
max        547.488298
Name: popularity, dtype: float64

In [38]:
type(dff['popularity'].iloc[0])

numpy.float64

In [39]:
dff['popularity'] = [float(s) for s in dff['popularity']]

In [40]:
dff['popularity'].describe()

count    46275.000000
mean         2.914243
std          5.968169
min          0.000000
25%          0.392888
50%          1.129734
75%          3.660012
max        547.488298
Name: popularity, dtype: float64

In [41]:
dff = dff[dff['popularity'] >= 2.9]

In [42]:
dff.head()

,adult,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,overview,...,spoken_languages,status,tagline,title,video,vote_average,vote_count,cast,Director,keywords
0,0,"{'id': 10194, 'name': 'Toy Story Collection', ...",30000000,Animation Comedy Family,http://toystory.disney.com/toy-story,862,tt0114709,en,Toy Story,"Led by Woody, Andy's toys live happily in his ...",...,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,NaN,Toy Story,False,7.7,5415.0,Tom Hanks Tim Allen Don Rickles Jim Varney Wal...,John Lasseter,jealousy toy boy friendship friends rivalry bo...
1,0,NaN,65000000,Adventure Fantasy Family,NaN,8844,tt0113497,en,Jumanji,When siblings Judy and Peter discover an encha...,...,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,Roll the dice and unleash the excitement!,Jumanji,False,6.9,2413.0,Robin Williams Jonathan Hyde Kirsten Dunst Bra...,Joe Johnston,board game disappearance based on children's b...
2,0,"{'id': 119050, 'name': 'Grumpy Old Men Collect...",0,Romance Comedy,NaN,15602,tt0113228,en,Grumpier Old Men,A family wedding reignites the ancient feud be...,...,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Still Yelling. Still Fighting. Still Ready for...,Grumpier Old Men,False,6.5,92.0,Walter Matthau Jack Lemmon Ann-Margret Sophia ...,Howard Deutch,fishing best friend duringcreditsstinger old men
3,0,NaN,16000000,Comedy Drama Romance,NaN,31357,tt0114885,en,Waiting to Exhale,"Cheated on, mistreated and stepped on, the wom...",...,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Friends are the people who let you be yourself...,Waiting to Exhale,False,6.1,34.0,Whitney Houston Angela Bassett Loretta Devine ...,Forest Whitaker,based on novel interracial relationship single...
4,0,"{'id': 96871, 'name': 'Father of the Bride Col...",0,Comedy,NaN,11862,tt0113041,en,Father of the Bride Part II,Just when George Banks has recovered from his ...,...,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Just When His World Is Back To Normal... He's ...,Father of the Bride Part II,False,5.7,173.0,Steve Martin Diane Keaton Martin Short Kimberl...,Charles Shyer,baby midlife crisis confidence aging daughter ...


In [43]:
dff.shape

(13322, 27)

In [44]:
dff.isnull().sum()

adult                        0
belongs_to_collection    10841
budget                       0
genres                       0
homepage                  9893
id                           0
imdb_id                      0
original_language            0
original_title               0
overview                    34
popularity                   0
poster_path                  0
production_companies         0
production_countries         0
release_year                 0
revenue                      0
runtime                      0
spoken_languages             0
status                       0
tagline                   3771
title                        0
video                        0
vote_average                 0
vote_count                   0
cast                         0
Director                    15
keywords                     0
dtype: int64

In [45]:
dff = dff[dff['release_year'] >= 1960]
dff.head()

,adult,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,overview,...,spoken_languages,status,tagline,title,video,vote_average,vote_count,cast,Director,keywords
0,0,"{'id': 10194, 'name': 'Toy Story Collection', ...",30000000,Animation Comedy Family,http://toystory.disney.com/toy-story,862,tt0114709,en,Toy Story,"Led by Woody, Andy's toys live happily in his ...",...,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,NaN,Toy Story,False,7.7,5415.0,Tom Hanks Tim Allen Don Rickles Jim Varney Wal...,John Lasseter,jealousy toy boy friendship friends rivalry bo...
1,0,NaN,65000000,Adventure Fantasy Family,NaN,8844,tt0113497,en,Jumanji,When siblings Judy and Peter discover an encha...,...,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,Roll the dice and unleash the excitement!,Jumanji,False,6.9,2413.0,Robin Williams Jonathan Hyde Kirsten Dunst Bra...,Joe Johnston,board game disappearance based on children's b...
2,0,"{'id': 119050, 'name': 'Grumpy Old Men Collect...",0,Romance Comedy,NaN,15602,tt0113228,en,Grumpier Old Men,A family wedding reignites the ancient feud be...,...,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Still Yelling. Still Fighting. Still Ready for...,Grumpier Old Men,False,6.5,92.0,Walter Matthau Jack Lemmon Ann-Margret Sophia ...,Howard Deutch,fishing best friend duringcreditsstinger old men
3,0,NaN,16000000,Comedy Drama Romance,NaN,31357,tt0114885,en,Waiting to Exhale,"Cheated on, mistreated and stepped on, the wom...",...,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Friends are the people who let you be yourself...,Waiting to Exhale,False,6.1,34.0,Whitney Houston Angela Bassett Loretta Devine ...,Forest Whitaker,based on novel interracial relationship single...
4,0,"{'id': 96871, 'name': 'Father of the Bride Col...",0,Comedy,NaN,11862,tt0113041,en,Father of the Bride Part II,Just when George Banks has recovered from his ...,...,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Just When His World Is Back To Normal... He's ...,Father of the Bride Part II,False,5.7,173.0,Steve Martin Diane Keaton Martin Short Kimberl...,Charles Shyer,baby midlife crisis confidence aging daughter ...


In [46]:
dff.shape

(12597, 27)

In [47]:
dff['vote_average'].describe()

count    12597.000000
mean         6.171041
std          0.945011
min          0.000000
25%          5.600000
50%          6.200000
75%          6.800000
max         10.000000
Name: vote_average, dtype: float64

In [48]:
dff = dff[dff['vote_average'] >= 6.1]
dff.shape

(7237, 27)

In [49]:
dff.reset_index(inplace=True)


In [50]:
dff.head()

,index,adult,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,...,spoken_languages,status,tagline,title,video,vote_average,vote_count,cast,Director,keywords
0,0,0,"{'id': 10194, 'name': 'Toy Story Collection', ...",30000000,Animation Comedy Family,http://toystory.disney.com/toy-story,862,tt0114709,en,Toy Story,...,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,NaN,Toy Story,False,7.7,5415.0,Tom Hanks Tim Allen Don Rickles Jim Varney Wal...,John Lasseter,jealousy toy boy friendship friends rivalry bo...
1,1,0,NaN,65000000,Adventure Fantasy Family,NaN,8844,tt0113497,en,Jumanji,...,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,Roll the dice and unleash the excitement!,Jumanji,False,6.9,2413.0,Robin Williams Jonathan Hyde Kirsten Dunst Bra...,Joe Johnston,board game disappearance based on children's b...
2,2,0,"{'id': 119050, 'name': 'Grumpy Old Men Collect...",0,Romance Comedy,NaN,15602,tt0113228,en,Grumpier Old Men,...,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Still Yelling. Still Fighting. Still Ready for...,Grumpier Old Men,False,6.5,92.0,Walter Matthau Jack Lemmon Ann-Margret Sophia ...,Howard Deutch,fishing best friend duringcreditsstinger old men
3,3,0,NaN,16000000,Comedy Drama Romance,NaN,31357,tt0114885,en,Waiting to Exhale,...,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Friends are the people who let you be yourself...,Waiting to Exhale,False,6.1,34.0,Whitney Houston Angela Bassett Loretta Devine ...,Forest Whitaker,based on novel interracial relationship single...
4,5,0,NaN,60000000,Action Crime Drama Thriller,NaN,949,tt0113277,en,Heat,...,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,A Los Angeles Crime Saga,Heat,False,7.7,1886.0,Al Pacino Robert De Niro Val Kilmer Jon Voight...,Michael Mann,robbery detective bank obsession chase shootin...


In [51]:
dff = dff.drop(['index'],axis = 1)

In [52]:
dff.head()

,adult,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,overview,...,spoken_languages,status,tagline,title,video,vote_average,vote_count,cast,Director,keywords
0,0,"{'id': 10194, 'name': 'Toy Story Collection', ...",30000000,Animation Comedy Family,http://toystory.disney.com/toy-story,862,tt0114709,en,Toy Story,"Led by Woody, Andy's toys live happily in his ...",...,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,NaN,Toy Story,False,7.7,5415.0,Tom Hanks Tim Allen Don Rickles Jim Varney Wal...,John Lasseter,jealousy toy boy friendship friends rivalry bo...
1,0,NaN,65000000,Adventure Fantasy Family,NaN,8844,tt0113497,en,Jumanji,When siblings Judy and Peter discover an encha...,...,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,Roll the dice and unleash the excitement!,Jumanji,False,6.9,2413.0,Robin Williams Jonathan Hyde Kirsten Dunst Bra...,Joe Johnston,board game disappearance based on children's b...
2,0,"{'id': 119050, 'name': 'Grumpy Old Men Collect...",0,Romance Comedy,NaN,15602,tt0113228,en,Grumpier Old Men,A family wedding reignites the ancient feud be...,...,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Still Yelling. Still Fighting. Still Ready for...,Grumpier Old Men,False,6.5,92.0,Walter Matthau Jack Lemmon Ann-Margret Sophia ...,Howard Deutch,fishing best friend duringcreditsstinger old men
3,0,NaN,16000000,Comedy Drama Romance,NaN,31357,tt0114885,en,Waiting to Exhale,"Cheated on, mistreated and stepped on, the wom...",...,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Friends are the people who let you be yourself...,Waiting to Exhale,False,6.1,34.0,Whitney Houston Angela Bassett Loretta Devine ...,Forest Whitaker,based on novel interracial relationship single...
4,0,NaN,60000000,Action Crime Drama Thriller,NaN,949,tt0113277,en,Heat,"Obsessive master thief, Neil McCauley leads a ...",...,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,A Los Angeles Crime Saga,Heat,False,7.7,1886.0,Al Pacino Robert De Niro Val Kilmer Jon Voight...,Michael Mann,robbery detective bank obsession chase shootin...


In [53]:
selected_features = ['genres','keywords','tagline','cast','Director']
for feature in selected_features:
    dff[feature] = dff[feature].fillna('')

In [54]:
combined_features = dff['genres']+' '+ dff['keywords']+' '+dff['tagline']+' '+dff['cast']+' '+dff['Director']
print(combined_features)

0       Animation Comedy Family jealousy toy boy frien...
1       Adventure Fantasy Family board game disappeara...
2       Romance Comedy fishing best friend duringcredi...
3       Comedy Drama Romance based on novel interracia...
4       Action Crime Drama Thriller robbery detective ...
                              ...                        
7232    TV Movie Family Action Comedy Music Adventure ...
7233    Comedy holiday nudist camp camping tent field ...
7234    Animation superhero lego  Grey Griffin Tara St...
7235    Romance Comedy  Likes are easy, love is hard. ...
7236    Family Animation Romance Comedy love teenager ...
Length: 7237, dtype: object


In [55]:
vectorizer = TfidfVectorizer()
feature_vectors = vectorizer.fit_transform(combined_features)
print(feature_vectors)

  (0, 30125)	0.17947239793798447
  (0, 26090)	0.189303239930725
  (0, 40786)	0.13471877998812265
  (0, 18529)	0.1230455958795329
  (0, 46335)	0.1022500224374359
  (0, 16346)	0.1740620951750553
  (0, 30495)	0.07743579374954367
  (0, 35290)	0.1647761264416639
  (0, 30239)	0.14045022439489147
  (0, 13640)	0.20069887255787774
  (0, 55987)	0.11682120763923051
  (0, 16321)	0.12982436561600424
  (0, 36598)	0.12488793799538213
  (0, 42120)	0.1775316952829986
  (0, 2106)	0.13317393656968105
  (0, 43514)	0.15725615978497337
  (0, 26229)	0.16407173310482182
  (0, 47973)	0.12096880770558113
  (0, 56259)	0.12553453203774334
  (0, 55163)	0.20610917532080691
  (0, 26094)	0.09122295473035437
  (0, 44358)	0.17249730454064308
  (0, 14435)	0.09269948229079358
  (0, 1383)	0.10071137287255841
  (0, 53067)	0.09175938899314791
  :	:
  (7235, 56837)	0.11848970348515478
  (7235, 2503)	0.09047981604982978
  (7235, 22092)	0.10483741290343446
  (7235, 31826)	0.06442660640438741
  (7235, 44916)	0.05385007192976492

In [56]:
similarity = cosine_similarity(feature_vectors)

In [57]:
similarity.shape

(7237, 7237)

In [58]:
movie_name = input(' Enter your favourite movie name : ')

 Enter your favourite movie name : Avatar


In [59]:
list_of_all_titles = dff['title'].tolist()
print(list_of_all_titles)

['Toy Story', 'Jumanji', 'Grumpier Old Men', 'Waiting to Exhale', 'Heat', 'Sabrina', 'GoldenEye', 'The American President', 'Balto', 'Nixon', 'Casino', 'Sense and Sensibility', 'Four Rooms', 'Ace Ventura: When Nature Calls', 'Get Shorty', 'Copycat', 'Powder', 'Leaving Las Vegas', 'Now and Then', 'The City of Lost Children', 'Dangerous Minds', 'Twelve Monkeys', 'Dead Man Walking', 'It Takes Two', 'Clueless', 'Richard III', 'Dead Presidents', 'Restoration', 'To Die For', 'How To Make An American Quilt', 'Se7en', 'Pocahontas', 'The Usual Suspects', 'Mighty Aphrodite', 'Georgia', 'Home for the Holidays', 'The Postman', "Mr. Holland's Opus", "Don't Be a Menace to South Central While Drinking Your Juice in the Hood", 'Friday', 'From Dusk Till Dawn', 'Kicking and Screaming', 'Screamers', 'The Crossing Guard', "Things to Do in Denver When You're Dead", 'White Squall', 'Nick of Time', 'Beautiful Girls', 'La Haine', 'Bottle Rocket', 'Happy Gilmore', 'The Bridges of Madison County', 'Muppet Treas

In [60]:
find_close_match = difflib.get_close_matches(movie_name, list_of_all_titles)
print(find_close_match)

['Avatar', 'Agatha', 'Matador']


In [61]:
close_match = find_close_match[0]
print(close_match)

Avatar


In [72]:

index_of_the_movie = dff[dff.title == close_match].index.values[0]
print(index_of_the_movie)

4268


In [73]:
similarity_score = list(enumerate(similarity[index_of_the_movie]))
print(similarity_score)

[(0, 0.0), (1, 0.032220522314020464), (2, 0.008406618814091054), (3, 0.007107810358186855), (4, 0.03748609639509114), (5, 0.03156414333917127), (6, 0.012626730208101032), (7, 0.01702065509363394), (8, 0.004529066417260006), (9, 0.03885270526773085), (10, 0.03241617798264117), (11, 0.016994670015488333), (12, 0.03021351022768045), (13, 0.005520702729211508), (14, 0.03576964505534839), (15, 0.025409517894949822), (16, 0.03787173851304977), (17, 0.024210838455675475), (18, 0.030586424513479652), (19, 0.015326472098089038), (20, 0.009489171486972975), (21, 0.05713646801656406), (22, 0.04042065213387246), (23, 0.022374425237662112), (24, 0.02697017376399262), (25, 0.021146161907335658), (26, 0.01742679284471345), (27, 0.012859491197506213), (28, 0.03949444027544963), (29, 0.032526891977463), (30, 0.03950866518442656), (31, 0.04339050362220252), (32, 0.014907681886349087), (33, 0.023972335971554008), (34, 0.009469817487652335), (35, 0.025078816698641877), (36, 0.0028923323787912792), (37, 0.

In [74]:
len(similarity_score)

7237

In [76]:
sorted_similar_movies = sorted(similarity_score, key = lambda x:x[1], reverse = True) 
print(sorted_similar_movies)

[(4268, 0.9999999999999994), (433, 0.14891549705601384), (422, 0.14298597274669522), (508, 0.13579797304152494), (2829, 0.12716289137709325), (6940, 0.1256789551166741), (2572, 0.11856943167561737), (1944, 0.11533957641174068), (2075, 0.11486968371889318), (5204, 0.11402357875721289), (6035, 0.1128863085402941), (593, 0.11064375209485348), (857, 0.10822822484053891), (693, 0.10551607559898325), (5310, 0.10449297882651336), (840, 0.10084080000429921), (4443, 0.09819665002632216), (6946, 0.09652655741274169), (1430, 0.09405560906651038), (7052, 0.09391187905648649), (741, 0.09295563834576608), (3927, 0.09230126513916119), (4112, 0.09188874579119675), (1512, 0.09091613171071909), (4853, 0.0904143324297987), (418, 0.08972785510629054), (3305, 0.08965050111134197), (580, 0.08738716763404318), (2549, 0.0872803828781417), (218, 0.08703959836600716), (1132, 0.08697326478096834), (3499, 0.08550781244033151), (1195, 0.08546371593656044), (1402, 0.08447488823181622), (2522, 0.08406609855614612), 

In [77]:
print('Movies suggested for you : \n')

i = 1

for movie in sorted_similar_movies:
    index = movie[0]
    title_from_index = dff[dff.index==index]['title'].values[0]
    if (i<30):
        print(i, '.',title_from_index)
        i+=1

Movies suggested for you : 

1 . Avatar
2 . Alien
3 . Aliens
4 . Alien³
5 . Journey to the Far Side of the Sun
6 . Rogue One: A Star Wars Story
7 . Babylon 5: A Call to Arms
8 . Silent Running
9 . Treasure Planet
10 . Star Trek Into Darkness
11 . Home
12 . Event Horizon
13 . Soldier
14 . Armageddon
15 . Gravity
16 . The Thing
17 . Hubble 3D
18 . Passengers
19 . Space Cowboys
20 . Life
21 . Dune
22 . Stargate: Continuum
23 . Star Trek
24 . Gettysburg
25 . John Carter
26 . The Empire Strikes Back
27 . Star Wreck: In the Pirkinning
28 . The Fifth Element
29 . Babylon 5: In the Beginning
